# Acute Coronary Syndrome Study

Lets see if we can run [the queries](http://build.fhir.org/ig/HL7/vulcan-rwd/acs.html) for cohort criteria.

See:
- https://clinicaltrials.gov/ct2/show/NCT02190123
- https://confluence.hl7.org/display/FHIR/Public+Test+Servers